In [ ]:
from dotenv import load_dotenv
import sys

load_dotenv()
sys.path.append("../")
# from graph1 import graph

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [15]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage


class AlignmentState(BaseModel):
    detailed_description: str = Field(description="what you want to say")


# 如果你的第三方大模型兼容 OpenAI Completion 接口，可以这样用:
llm = ChatOpenAI(
    model_name="Qwen/QwQ-32B",  # 或者换成你对应的模型
    temperature=0.7,
    base_url="https://api.siliconflow.cn/v1",
    api_key="sk-isdgjofhetjtwirzjjcrbaidzsonydgkmzfdqquymflnuaxh",
)
llm = llm.with_structured_output(AlignmentState)
res = llm.invoke(
    [
        SystemMessage(content="who you are?"),
    ]
)
res.detailed_description

'I am Qwen, a large language model developed by Alibaba Cloud. I have the ability to answer questions, create text such as stories, emails, scripts, and more. I can also perform logical reasoning, programming, express opinions, and play games. My capabilities are designed to assist users in a variety of tasks and provide engaging, informative responses.'